In [184]:
import sys
sys.path

['e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages\\paddle\\fluid\\..\\libs',
 'E:\\deep\\paddlepaddle_project\\paddle重写房价预测',
 'e:\\python\\anaconda3\\envs\\paddle\\python37.zip',
 'e:\\python\\anaconda3\\envs\\paddle\\DLLs',
 'e:\\python\\anaconda3\\envs\\paddle\\lib',
 'e:\\python\\anaconda3\\envs\\paddle',
 '',
 'e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages',
 'e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages\\win32',
 'e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages\\win32\\lib',
 'e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages\\Pythonwin',
 'e:\\python\\anaconda3\\envs\\paddle\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\魏宏炜\\.ipython']

### 用paddle重写波士顿房价预测问题
### 数据集为housing.data文件
###### 数据中的标签：[ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
###### 前面13个表示影响房价的指标，最后一个为房价中位数。

In [1]:
#调用使用的模块
import paddle
import paddle.fluid as fluid
import os
import paddle.fluid.dygraph as dygraph#飞桨动态图
from paddle.fluid.dygraph import Linear#提供诸多神经网络的基础模块
import numpy as np
import pandas as pd
import random

In [2]:

#数据处理部分
def load_data():
    # 从文件导入数据
    datafile = 'housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    #将feature格式转换为（样本数，特征数）格式
    feature = [data[k:k+feature_num] for k in range(0,len(data),feature_num)]
    feature = np.array(feature)
    #输出（506，14）
    #print(feature.shape)
    
    #划分训练集合，测试集,前面80%样本作为训练集，后面20%样本作为测试集
    offset = int(feature.shape[0] * 0.8)
    train = feature[:offset]
    test = feature[offset:]
    
    #归一化数据
    maximum ,minimum , average = train.max(axis = 0) ,train.min(axis = 0) ,\
                        train.sum(axis = 0) / train.shape[0]
    global max_values
    global min_values
    global avg_values
    max_values = maximum
    min_values = minimum
    avg_values = average
    
    for i in range(train.shape[1]):
        train[:,i] = (train[:,i] - average[i] ) / (maximum[i] - minimum[i])
    '''
    train = train[:,:-1]
    train_y = train[:,-1]
    '''
    for i in range(test.shape[1] - 1):
        test[:,i] = (test[:,i] - average[i] ) / (maximum[i] - minimum[i])
    
    '''
    test_x = test[:,:-1]
    test_y = test[:,-1]
    '''
    
    data = {
        "train":train,
        "test":test
    }
    
    return data


In [3]:
#定义网络结构：
class Regressor(fluid.dygraph.Layer):
    def __init__(self):
        super(Regressor,self).__init__()
        #定义一层全连接层，输出维度是1，激活函数为None，即不使用激活函数
        self.fc = Linear(input_dim = 13,output_dim = 1,act = None)
        
    def forward(self,inputs):#定义前向传播函数
        x = self.fc(inputs)
        return x
    def backward():
        pass
        


In [4]:
#训练配置
with fluid.dygraph.guard():
    #声明定义好的回归i模型
    model = Regressor()
    #运行模型
    model.train()
    #加载数据
    data = load_data()
    train = data["train"]
    
    #print(train.shape[0],len(data["test"]))
    #定义优化器sgd
    opt = fluid.optimizer.SGD(learning_rate = 0.01,parameter_list = model.parameters())
    
    

In [7]:
#训练过程
use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
with fluid.dygraph.guard(place):
    EPOCH_NUM = 10#训练轮数
    BATCH_SIZE = 10
    
    #定义外层循环
    for epoch_id in range(EPOCH_NUM):
        
        #随机打乱数据
        random.seed(1)
        np.random.shuffle(train)
        #将数据进行分割成多个mini_batch
        mini_batchs = [train[k:k+BATCH_SIZE] for k in range(0,train.shape[0],BATCH_SIZE)  ]
        #定义内层循环
        for iter_id,mini_batch in enumerate(mini_batchs):
            #获取数据集
            x = np.array(mini_batch[:, :-1]).astype('float32') # 获得当前批次训练数据
            y = np.array(mini_batch[:, -1:]).astype('float32') # 获得当前批次训练标签（真实房价）
            #将numpy数组转化为Variable形式
            house_features = dygraph.to_variable(x)
            prices = dygraph.to_variable(y)
            #前向传播
            predicts = model(house_features)
            #计算loss,用均方误差
            loss = fluid.layers.square_error_cost(predicts,label = prices)
            avg_loss = fluid.layers.mean(loss)
            
            if iter_id%20==0:
                print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
            #计算反向过程
            avg_loss.backward()
            #最小化loss，更新参数
            opt.minimize(avg_loss)

            #清除梯度
            model.clear_gradients()
        #保存模型
        fluid.save_dygraph(model.state_dict(),"LR_model")

epoch: 0, iter: 0, loss is: [0.00771079]
epoch: 0, iter: 20, loss is: [0.01681274]
epoch: 0, iter: 40, loss is: [0.00359185]
epoch: 1, iter: 0, loss is: [0.00543477]
epoch: 1, iter: 20, loss is: [0.0055231]
epoch: 1, iter: 40, loss is: [0.01899833]
epoch: 2, iter: 0, loss is: [0.00733241]
epoch: 2, iter: 20, loss is: [0.00674023]
epoch: 2, iter: 40, loss is: [0.00931837]
epoch: 3, iter: 0, loss is: [0.00520588]
epoch: 3, iter: 20, loss is: [0.00744813]
epoch: 3, iter: 40, loss is: [0.07268187]
epoch: 4, iter: 0, loss is: [0.03688293]
epoch: 4, iter: 20, loss is: [0.01200454]
epoch: 4, iter: 40, loss is: [0.00291833]
epoch: 5, iter: 0, loss is: [0.03325633]
epoch: 5, iter: 20, loss is: [0.01469778]
epoch: 5, iter: 40, loss is: [0.00163914]
epoch: 6, iter: 0, loss is: [0.0051043]
epoch: 6, iter: 20, loss is: [0.00453827]
epoch: 6, iter: 40, loss is: [0.00268995]
epoch: 7, iter: 0, loss is: [0.01046379]
epoch: 7, iter: 20, loss is: [0.00344645]
epoch: 7, iter: 40, loss is: [0.00556711]
ep

In [190]:
#测试模型
def load_one_example(data_dir):
    f = open(data_dir,"r")
    datas = f.readlines()
    #选择最后一条数据作为测试数据
    one_e = datas[-2].strip().split()
    one_data = [float(k) for k in one_e]
    
    #对数据进行归一化处理，房价不需要
    for i in range(len(one_data)-1):
        one_data[i] = (one_data[i]-avg_values[i]) / (max_values[i] - min_values[i])
    data = np.reshape(np.array(one_data[0:-1]),[1,-1]).astype(np.float32)
    
    label = one_data[-1]
    #print(data)
    return data,label


In [191]:
with dygraph.guard():
    #加载训练好的参数
    #第一句是从文件中读取模型参数；第二句是将参数内容加载到模型。
    model_dict,_ = fluid.load_dygraph("LR_model")
    model.load_dict(model_dict)
    #将模型的状态调整为eval()（校验）
    model.eval()
    
    #获取测试集
    test_data , label = load_one_example("housing.data")
    #将数据转换为动态图的variable模式
    test_data = dygraph.to_variable(test_data)
    results = model(test_data)
    
    #对输出结果进行返归一化处理
    results = results*(max_values[-1] - min_values[-1]) + avg_values[-1]
    print("预测房价：",results.numpy().squeeze())
    print("真实房价：",label)
    
    

预测房价： 26.753998
真实房价： 22.0
